#Thinkful Data Science Course
##Unit 4: Predicting the Future; Lesson 2: Decision Trees
##Random Forest Analysis

###Data Cleaning and Exploration

Download and unzip UCI HAR Dataset.zip from the UCI Machine Learning Repository (click "Data Folder" next to "Download").
Read through features_info.txt. This file describes each feature, its physical significance, and also describes features that are derived from raw data by doing some averaging, or sampling, or some operation that gives a numerical result. What do you notice about the dataset? What kind of guidelines for approaching the dataset do you find?
In static activities (sit, stand, lie down) motion information will not be very useful.
In the dynamic activities (3 types of walking) motion will be significant.
Angle variables will be useful both in differentiating “lie vs stand” and “walk up vs walk down”.
Acceleration and Jerk variables are important in distinguishing various kinds of motion while filtering out random tremors while static.
Mag and Angle variables contain the same info as (e.g., are strongly correlated with) XYZ variables. We choose to focus on the latter as they are simpler to reason about. This is a very important point to understand as it results in elimination of a few hundred variables.
We ignore the band variables as we have no simple way to interpret the meaning and relate them to physical activities.
mean and std are important, skewness and kurtosis may also so we include all of these.

####Note: 
for each of the tasks below, think about how you will fix the data and what the implications/ramifications of your fixes will be. Also, can you think of a way to accomplish several of the cleaning tasks at once?

Identify and fix the inclusion of ( ) in column names.
Identify and fix the inclusion of ‘-’ in column names.
Replace ',' with '_' in column names.
Drop 'Body' and 'Mag' from column names.
Map 'mean' and 'std' to 'Mean' and 'STD'
Make 'activity' a categorical variable.
Plot a histogram of Body Acceleration Magnitude to see how each variable does as a predictor of static versus dynamic activities.


In [4]:
import pandas as pd
# import randomforest as rf
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as skm
import pylab as pl

In [5]:
samtrain = pd.read_csv('/Users/molliepettit/Desktop/Data Science/Thinkful/Projects/Unit4/samsung/samtrain.csv')
samval = pd.read_csv('/Users/molliepettit/Desktop/Data Science/Thinkful/Projects/Unit4/samsung/samval.csv')
samtest = pd.read_csv('/Users/molliepettit/Desktop/Data Science/Thinkful/Projects/Unit4/samsung/samtest.csv')
samsungdata = pd.read_csv('/Users/molliepettit/Desktop/Data Science/Thinkful/Projects/Unit4/samsung/samsungdata.csv')
samsungmin = pd.read_csv('/Users/molliepettit/Desktop/Data Science/Thinkful/Projects/Unit4/samsung/samsungmin.csv')

We use the Python RandomForest package from the scikits.learn collection of algorithms. 

For this we need to convert the target column ('activity') to integer values because the Python RandomForest package requires that.  

We map activity to an integer according to laying = 1, sitting = 2, standing = 3, walk = 4, walkup = 5, walkdown = 6 

Code also uses library randomforest.py (uploaded in git)

In [6]:
samtrain

,Unnamed: 0,tAccMean,tAccStd,tJerkMean,tJerkSD,tGyroMean,tGyroSD,tGyroJerkMean,tGyroJerkMagSD,fAccMean,...,fGyroJerkKurtosis,angleAccGravity,angleJerkGravity,angleGyroGravity,angleGyroJerkGravity,angleXGravity,angleYGravity,angleZGravity,subject,activity
0,1,-0.959434,-0.950551,-0.993306,-0.994336,-0.968959,-0.964335,-0.994248,-0.991368,-0.952155,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,standing
1,2,-0.979289,-0.976057,-0.991253,-0.991694,-0.980683,-0.983754,-0.995123,-0.996102,-0.980857,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,standing
2,3,-0.983703,-0.988020,-0.988531,-0.990397,-0.976317,-0.986051,-0.993403,-0.995091,-0.987795,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,standing
3,4,-0.986542,-0.986421,-0.993078,-0.993381,-0.982060,-0.987351,-0.995502,-0.995267,-0.987519,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,standing
4,5,-0.992827,-0.991275,-0.993480,-0.995854,-0.985204,-0.989063,-0.995808,-0.995258,-0.993591,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,standing
5,6,-0.994295,-0.995249,-0.993018,-0.995424,-0.985894,-0.986440,-0.995275,-0.995205,-0.994836,...,-0.844619,0.082632,-0.143439,0.275041,-0.368224,-0.849632,0.184823,-0.042126,1,standing
6,7,-0.987466,-0.982746,-0.991314,-0.989448,-0.985501,-0.984625,-0.993719,-0.995269,-0.982135,...,-0.564430,-0.212754,-0.230622,0.014637,-0.189512,-0.852150,0.182170,-0.043010,1,standing
7,8,-0.977458,-0.975997,-0.990851,-0.990388,-0.984506,-0.986838,-0.994291,-0.995211,-0.982184,...,-0.421715,-0.020888,0.593996,-0.561871,0.467383,-0.851017,0.183779,-0.041976,1,standing
8,9,-0.983631,-0.977514,-0.993325,-0.992887,-0.989370,-0.990914,-0.995850,-0.995588,-0.983131,...,-0.572995,0.012954,0.080936,-0.234313,0.117797,-0.847971,0.188982,-0.037364,1,standing
9,10,-0.986772,-0.986702,-0.985388,-0.983643,-0.984524,-0.980264,-0.989799,-0.990735,-0.984994,...,0.140452,-0.020590,-0.127730,-0.482871,-0.070670,-0.848294,0.190310,-0.034417,1,standing


In [7]:
samval

,Unnamed: 0,tAccMean,tAccStd,tJerkMean,tJerkSD,tGyroMean,tGyroSD,tGyroJerkMean,tGyroJerkMagSD,fAccMean,...,fGyroJerkKurtosis,angleAccGravity,angleJerkGravity,angleGyroGravity,angleGyroJerkGravity,angleXGravity,angleYGravity,angleZGravity,subject,activity
0,3966,-0.782808,-0.746267,-0.836010,-0.670788,-0.706171,-0.690900,-0.805970,-0.778162,-0.705378,...,-0.440427,-0.015173,-0.289105,-0.588349,0.051690,-0.658107,0.306648,0.130139,21,standing
1,3967,-0.886897,-0.850645,-0.962290,-0.937674,-0.865987,-0.757075,-0.938773,-0.880460,-0.887307,...,0.384014,-0.457228,0.284724,-0.525611,0.834287,-0.631113,0.310777,0.159743,21,standing
2,3968,-0.900747,-0.872747,-0.964719,-0.941369,-0.877378,-0.836477,-0.964791,-0.956132,-0.907046,...,-0.086438,0.166681,0.057764,0.628871,-0.734367,-0.632521,0.305594,0.165904,21,standing
3,3969,-0.883321,-0.858033,-0.930145,-0.913145,-0.839208,-0.821157,-0.945937,-0.957287,-0.886470,...,-0.530977,0.075160,0.184325,-0.017363,-0.937170,-0.641088,0.304133,0.157569,21,standing
4,3970,-0.937834,-0.894600,-0.952980,-0.915333,-0.915617,-0.837527,-0.967671,-0.959835,-0.895751,...,0.020834,-0.099957,-0.017790,-0.241290,0.819461,-0.643430,0.306376,0.150995,21,standing
5,3971,-0.967582,-0.958965,-0.978520,-0.962604,-0.956284,-0.950400,-0.982568,-0.977330,-0.968196,...,-0.629090,-0.007712,0.064342,0.467554,0.442353,-0.646012,0.307363,0.145917,21,standing
6,3972,-0.975128,-0.953184,-0.978236,-0.963259,-0.969381,-0.943871,-0.984932,-0.976879,-0.965543,...,-0.364225,-0.013597,-0.570739,0.470380,-0.420999,-0.650765,0.302348,0.148019,21,standing
7,3973,-0.991612,-0.992505,-0.989447,-0.990300,-0.987040,-0.990144,-0.994072,-0.995465,-0.990847,...,-0.838405,-0.131150,-0.627706,-0.897099,-0.783737,-0.652798,0.299634,0.149794,21,standing
8,3974,-0.992257,-0.993841,-0.992208,-0.990973,-0.988556,-0.988641,-0.996521,-0.997919,-0.992129,...,-0.867300,-0.030527,0.247046,-0.691627,-0.248657,-0.654532,0.298122,0.149988,21,standing
9,3975,-0.994733,-0.995311,-0.993299,-0.996857,-0.992795,-0.990051,-0.997342,-0.998594,-0.994516,...,-0.924335,-0.056554,0.237799,-0.663555,0.863976,-0.653458,0.298277,0.151115,21,standing


In [8]:
import randomforests as rf

ImportError: No module named 'randomforests'